In [0]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
data_file = open('intents2.json').read()
intents = json.loads(data_file)

In [0]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        print("w",w)
        words.extend(w)
        print("words",words)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        print("documnets",documents)
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        print("classes",classes)

w ['Hi', 'there']
words ['Hi', 'there']
documnets [(['Hi', 'there'], 'greeting')]
classes ['greeting']
w ['How', 'are', 'you']
words ['Hi', 'there', 'How', 'are', 'you']
documnets [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting')]
classes ['greeting']
w ['Is', 'anyone', 'there', '?']
words ['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?']
documnets [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting')]
classes ['greeting']
w ['Hey']
words ['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey']
documnets [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting')]
classes ['greeting']
w ['Hola']
words ['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola']
documnets [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greetin

In [0]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
print("1",words)
words = sorted(list(set(words)))
print("2",words)
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

1 ['hi', 'there', 'how', 'are', 'you', 'is', 'anyone', 'there', 'hey', 'hola', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'goodbye', 'nice', 'chatting', 'to', 'you', ',', 'bye', 'till', 'next', 'time', 'thanks', 'thank', 'you', 'that', "'s", 'helpful', 'awesome', ',', 'thanks', 'thanks', 'for', 'helping', 'me', 'how', 'you', 'could', 'help', 'me', 'what', 'you', 'can', 'do', 'what', 'help', 'you', 'provide', 'how', 'you', 'can', 'be', 'helpful', 'what', 'support', 'is', 'offered', 'how', 'to', 'check', 'adverse', 'drug', 'reaction', 'open', 'adverse', 'drug', 'module', 'give', 'me', 'a', 'list', 'of', 'drug', 'causing', 'adverse', 'behavior', 'list', 'all', 'drug', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'which', 'drug', 'dont', 'have', 'adverse', 'reaction', 'open', 'blood', 'pressure', 'module', 'task', 'related', 'to', 'blood', 'pressure', 'blood', 'pressure', 'data', 'entry', 'i', 'want', 'to', 'log', 'blood', 'pressure', 'result', 'blood', 'pressure

In [0]:
words

["'s",
 ',',
 '2011',
 'a',
 'about',
 'adverse',
 'all',
 'an',
 'anyone',
 'are',
 'awesome',
 'be',
 'behavior',
 'best',
 'blood',
 'by',
 'bye',
 'can',
 'captian',
 'causing',
 'chatting',
 'check',
 'could',
 'country',
 'cup',
 'data',
 'day',
 'detail',
 'do',
 'dont',
 'drug',
 'entry',
 'find',
 'for',
 'give',
 'good',
 'goodbye',
 'have',
 'hello',
 'help',
 'helpful',
 'helping',
 'hey',
 'hi',
 'history',
 'hola',
 'hospital',
 'how',
 'i',
 'icc',
 'id',
 'in',
 'india',
 'indian',
 'innovating',
 'innovation',
 'is',
 'later',
 'lift',
 'list',
 'load',
 'locate',
 'located',
 'log',
 'looking',
 'lookup',
 'management',
 'me',
 'module',
 'mumbai',
 'nearby',
 'next',
 'nice',
 'of',
 'offered',
 'only',
 'open',
 'patient',
 'pharmacy',
 'pressure',
 'provide',
 'reaction',
 'related',
 'result',
 'search',
 'searching',
 'see',
 'show',
 'suitable',
 'support',
 'task',
 'tell',
 'thank',
 'thanks',
 'that',
 'the',
 'there',
 'till',
 'time',
 'to',
 'transfer',
 '

In [0]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
print(output_empty)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    print("p1",pattern_words)
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    print("p2",pattern_words)
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
print("randaom",training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
print("train_x",train_x)
train_y = list(training[:,1])
print("train_y",train_y)
print("Training data created")


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
p1 ['Hi', 'there']
p2 ['hi', 'there']
p1 ['How', 'are', 'you']
p2 ['how', 'are', 'you']
p1 ['Is', 'anyone', 'there', '?']
p2 ['is', 'anyone', 'there', '?']
p1 ['Hey']
p2 ['hey']
p1 ['Hola']
p2 ['hola']
p1 ['Hello']
p2 ['hello']
p1 ['Good', 'day']
p2 ['good', 'day']
p1 ['Bye']
p2 ['bye']
p1 ['See', 'you', 'later']
p2 ['see', 'you', 'later']
p1 ['Goodbye']
p2 ['goodbye']
p1 ['Nice', 'chatting', 'to', 'you', ',', 'bye']
p2 ['nice', 'chatting', 'to', 'you', ',', 'bye']
p1 ['Till', 'next', 'time']
p2 ['till', 'next', 'time']
p1 ['Thanks']
p2 ['thanks']
p1 ['Thank', 'you']
p2 ['thank', 'you']
p1 ['That', "'s", 'helpful']
p2 ['that', "'s", 'helpful']
p1 ['Awesome', ',', 'thanks']
p2 ['awesome', ',', 'thanks']
p1 ['Thanks', 'for', 'helping', 'me']
p2 ['thanks', 'for', 'helping', 'me']
p1 ['How', 'you', 'could', 'help', 'me', '?']
p2 ['how', 'you', 'could', 'help', 'me', '?']
p1 ['What', 'you', 'can', 'do', '?']
p2 ['what', 'you', 'can', 'do', '?']
p1 ['W

In [0]:
maxLen = len(train_x)

In [0]:
#Another Model 
#from keras.models import Sequential, Model
# from keras.layers.embeddings import Embedding
# from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM
# max_question_len = 20
# batch_size = 32
# #Example of a Placeholder
# #input_sequence = Input((max_story_len,))
# question = Input((max_question_len,))
# #Create input encoder A:
# input_encoder_m = Sequential()
# input_encoder_m.add(Embedding(input_dim=len(words)+1,output_dim = 64)) 
# input_encoder_m.add(Dropout(0.3))
# #Outputs: (Samples, story_maxlen,embedding_dim) -- Gives a list of #the lenght of the samples where each item has the
# #lenght of the max story lenght and every word is embedded in the embbeding dimension
# match = dot([input_encoded_m,question_encoded], axes = (2,2))
# match = Activation('softmax')(match)
# response = add([match,input_encoded_c])
# response = Permute((2,1))(response)
# answer = concatenate([response, question_encoded])
# answer = LSTM(32)(answer)
# answer = Dropout(0.5)(answer)
# #Output layer:
# answer = Dense(vocab_len)(answer) 
# #Output shape: (Samples, Vocab_size) #Yes or no and all 0s
# answer = Activation('softmax')(answer)
# model = Model([input_sequence,question], answer)
# model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [0]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [0]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1,validation_split=0.3)

Train on 39 samples, validate on 18 samples
Epoch 1/200
39/39 [==============================] - 0s 3ms/step - loss: 2.6485 - accuracy: 0.0513 - val_loss: 2.5552 - val_accuracy: 0.0556
Epoch 2/200
39/39 [==============================] - 0s 448us/step - loss: 2.5014 - accuracy: 0.1538 - val_loss: 2.5342 - val_accuracy: 0.0556
Epoch 3/200
39/39 [==============================] - 0s 437us/step - loss: 2.4880 - accuracy: 0.2308 - val_loss: 2.5157 - val_accuracy: 0.1111
Epoch 4/200
39/39 [==============================] - 0s 425us/step - loss: 2.4149 - accuracy: 0.1538 - val_loss: 2.4795 - val_accuracy: 0.2222
Epoch 5/200
39/39 [==============================] - 0s 433us/step - loss: 2.3197 - accuracy: 0.2308 - val_loss: 2.4477 - val_accuracy: 0.3333
Epoch 6/200
39/39 [==============================] - 0s 490us/step - loss: 2.2550 - accuracy: 0.3077 - val_loss: 2.4148 - val_accuracy: 0.3333
Epoch 7/200
39/39 [==============================] - 0s 437us/step - loss: 2.2039 - accuracy: 0.3846

In [0]:
model.predict()